In [ ]:
'''
開發用:
掛載Google硬碟
安裝套件

生產固定流程:
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

"""

重點流程：

- 掛載google drive
- 安裝套件
- 引入套件
- 建置主程序(含line_bot_api跟 webhookhandler)
- 建立主程序API介面接口(這樣才能接收用戶訊息)


* 製作QuickReply的按鈕功能 (後續包含將多個按鈕封包 => List)
* 建立一個TextSendMessage，當用戶輸入特定關鍵字就會將按鈕封包給用戶
* 建立一個關鍵字字典以觸發QuickReply功能(搭配上面使用)
* 對用戶輸入的文字進行判斷(看是將用戶引導到QuickReply功能 ; 還是報錯)

* 對用戶的點擊訊息進行判斷，若有postback的類型，告訴handler如何處理回傳值


- 最後五個步驟需多練習


@handler.add(XX事件) => 記憶：當handler遇到XX事件時，定義一個功能教他如何處理該事件

"""

In [ ]:
"""
資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

"""

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
"""
引用套件
"""

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import LineBotApi, WebhookHandler

# 引用無效簽章錯誤
from linebot.exceptions import InvalidSignatureError

In [ ]:
"""
建置主程序

建置handler與 line_bot_api
"""

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("HPNN2U0LfdttfMl7b5yCkW/h9yzuxYOGW4NYDnJuBD4qXTY/UagvGJK3OKAnGwQwgA1tYG9lPNFt92N+j33osr3nQ59os0a/bD+usRNyUumvLs0IYnHTR6FkmhCAaQ8WkcKZq6a3AJlA4tfw9jxLtgdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("e20a1f0d6418dd3bebb261a9fb9374ab")

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
"""
QuickReply的概念：

QuickReply: 快速執行預設動作、快速回覆(猶如答題選項A, B, C, D一般，點擊即可)

另外QuickReply還有一個重要功能: 「按完就會消失」

***** 要單獨製作按鈕，並設定功能時，幾乎都會載入xxxxxAction的模組 *****

"""
# ***** 此章節的核心 *****


"""
先引入相關套件(可發現很多xxxxxAction)
"""
from linebot.models import (
    MessageAction, URIAction, PostbackAction, DatetimePickerAction, CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

"""
開始製作設定QuickReply的按鍵

1. 用戶點擊QR按鈕後，就會快速執行預設動作，然後按鈕消失
2. 命名通常都是xxxx功能的QuickReplyButton
  => 如 datetimeQuickReplyButton = QuickReplyButton()功能
    參數部份: QuickReplyButton(action = xxxxAction())
                        
                  常用的 xxxxAction 包括 MessageAction(), DatetimePickerAction(), CameraAction(), ....
如下有多例                      
"""

# 創建一個QuickReplyButton(action = MessageAction())的QRB，點擊後用戶會發送按鈕設定好的文字訊息
# label => 按鈕顯示的名稱
# text => 以用戶的角度講話
TextQuickReplyButton = QuickReplyButton(
    action = MessageAction(
        label = "按鈕所顯示的文字",
        text = "我是點擊後所發送的字"
    )
)


# 創建一個QuickReplyButton(action = DatetimePickerAction())的QRB，點擊後會跳出時間供用戶選擇
# label => 按鈕顯示的名稱
# data => 隱藏資訊給用戶
# mode => 給用戶輸入的畫面天數
DateQuickReplyButton = QuickReplyButton(
    action = DatetimePickerAction(
        label = "時間選擇",
        data = "Data1",                       
        mode = "date"
    )
)


# 創建一個QuickReplyButton(action = CameraAction())的QRB，點擊後會開起相機
# label => 按鈕顯示的名稱
CameraQuickReplyButton = QuickReplyButton(
    action = CameraAction(
        label = "開啟相機"
    )
)


# 創建一個QuickReplyButton(action = CameraRollAction())的QRB，點擊後會打開相簿
# label => 按鈕顯示的名稱
CameraRollQuickReplyButton = QuickReplyButton(
    action = CameraRollAction(
        label = "從相簿選擇照片"
    )
)


# 創建一個QuickReplyButton(action = LocationAction())的QRB，點擊後會引導用戶發送當前地理位置
# label => 按鈕顯示的名稱
LocationQuickReplyButton = QuickReplyButton(
    action = LocationAction(
        label = "發送當前地理位置"
    )
)


# 創建一個QuickReplyButton(action = PostbackAction())的QRB，點擊後，會以Postback事件回應Server
# label => 按鈕顯示的名稱
# data => 點擊後送出的資料 (可利用data做很多隱藏功能)
PostbackQuickReplyButton = QuickReplyButton(
    action = PostbackAction(
        label = "我是回傳事件",
        text = "回傳值點擊後要給用戶看的字",
        data = "神秘的回傳資料"
    )
)

In [ ]:
"""

以QuickReply封裝多個QuickReplyButton


製作QRB List：
1. 命名通常會是一個list名稱，如: QuickReplyList = QuickReply() 
2. QuickReply()的參數為items，通常是一個list，裡面存放多個QuickReplyButton

看以下例子

"""
# 創建一個QuickReplyList(items = [])的QRL
QuickReplyList = QuickReply(
                items = [TextQuickReplyButton, DateQuickReplyButton, CameraQuickReplyButton,
                    CameraRollQuickReplyButton, LocationQuickReplyButton, PostbackQuickReplyButton]
)

In [ ]:
"""

製作一個TextSendMessage，並將剛封裝的QuickReply放入

=> 當用戶激發特定關鍵字後，將封裝的QuickReplyList發送給用戶點選

"""
# 將quickReplyList, 放入TextSendMessage 
from linebot.models import TextSendMessage

"""
當客戶「輸入特定關鍵字要回覆給用戶的物件」 = TextSendMessaage(text="問用戶的問題", quick_reply=我們建立的QRBList，提供給用戶回答問題用的)
"""
quick_reply_text_send_message = TextSendMessage(text="發送問題給用戶，請用戶回答", quick_reply=QuickReplyList)

In [ ]:
"""

建立一個字典，當用戶輸入特定關鍵字時，就會回覆用戶上面的「關鍵字回覆物件」

"""

# 從linebot.models引入訊息模型，文字收取與文字寄發訊息，及影像訊息
from linebot.models import MessageEvent, TextSendMessage, TextMessage, ImageSendMessage

# 建立特定關鍵字的字典(當用戶輸入特定關鍵字就會返回對應的動作)
template_message_dict = {"@reply": quick_reply_text_send_message}

In [ ]:
"""

對用戶發送的文字進行判斷，看是否要呼叫QuickReplyButton給用戶點擊

"""

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
# 為了要出發字典裡面的消息，當用戶發文字，就會回到字典裡面找結果並發送回用戶

# 當收到用戶文字傳訊時，前台將如何處置？
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
  # 條件1: 當用戶行為.訊息.文字.找到("XXX"):
  if (event.message.text.find("@") != -1):
    
    # 業務回覆訊息
    line_bot_api.reply_message(event.reply_token, template_message_dict.get(event.message.text))
  
  # 否則:
  else:
    # 業務回覆訊息
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text="查無此關鍵字，請輸入@reply來獲得QRBList"))

In [ ]:
"""

當用戶點擊QuickReplyButton後，若是觸發到PostBackEvent，則要對其回傳做相對應處理

"""
# ***** 重要 *****
# 遇到用戶觸發「回傳動作」我們的處理反應


from linebot.models import PostbackEvent

# PostBackEvent很重要，結合QuickReply可做好玩的事情(如不可逆的流程，即不可重複點擊)

#　記憶：
# 當handler
# add(遇到)
# (xxxxxEvent):時
# 就執行下列方法   =>   如同你要教前台小秘書該怎麼處理遇到的狀況

@handler.add(PostbackEvent)
def handle_post_messaage(event):
  # 先交給業務紀錄用戶個資 (個人資料 = 業務.紀錄用戶訊息(訊息來源的發送人的ID))
  user_profile = line_bot_api.get_profile(event.source.user_id)

  # 條件1: 當返回的資料中，資料等於XXX時，執行此條件
  # (如果 用戶行為.返回.資料中.找到("資料欄位") == XX時)
  if (event.postback.data.find("Data1") == 0):
    with open("user_profile_business.txt", "a") as myfile:
      myfile.write(json.dunp(vars(user_profile), sort_keys=True))
      myfile.write("\n")
      
      # 業務回覆訊息
      line_bot_api.reply_message(event.relpy_token, TextSendMessage(text="請稍後，將有專人為您服務"))

  # 條件2: 當上述條件行不通時，換試試看此條件
  elif (event.postback.data.find("Data2") == 0):
    with open("user_profile_business.txt", "a") as myfile:
      myfile.write(json.dump(vars(user_profile)), sort_keys=True)
      myfile.write("\n")
      
      # 業務回覆訊息
      line_bot_api.reply_message(event.reply_token, TextSendMessage(text="請稍後，將派專家協助您"))

  # 條件都不合時
  else:
    pass

In [ ]:
# 主程序運行
app.run()